## ML model `demo`

Predicts the type of iris plant.


### Graph

In [ ]:
from flypipe.examples.pipeline.model.demo.graph import graph
displayHTML(graph.html())

### How to train?

In [ ]:
import os
import mlflow
from flypipe.examples.pipeline.model.demo.train.evaluate import evaluate
from flypipe.examples.pipeline.model.demo.config import _config

ARTIFACT_LOCATION = _config["artifact_location"]
EXPERIMENT_PATH = _config["experiment_path"]
EXPERIMENT_NAME = _config["experiment_name"]


experiment_name = os.path.join(EXPERIMENT_PATH, EXPERIMENT_NAME)
# Ends any actve mlflow run
try:
    mlflow.end_run()
except Exception as e:
    pass

"""
Creates or gets an experiment from /Shared folder
Sets the artifact location to the mounted blob
"""
os.makedirs(ARTIFACT_LOCATION, exist_ok=True)

try:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME,
                                             artifact_location=ARTIFACT_LOCATION.replace("/dbfs/", "dbfs:/"))
except Exception as e:
    pass
finally:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    experiment_id = experiment.experiment_id

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print(f"Experiment: {experiment}")

"""
Starts the mlflow run with the experiment
"""
mlflow.start_run(experiment_id=experiment_id)
RUN_ID = mlflow.active_run().info.run_id
print(f"Active run_id: {RUN_ID}")

"""
Run train/evaluate graph
"""
df = evaluate.run()
display(df)

mlflow.end_run()

### How to run predictions?

> Note: as `config` node contains information of the run used for production, the code bellow can be simplified as

``` py

from flypipe.examples.pipeline.model.demo.predict.predict import predict
df = predict.run()
display(df)

```

In [ ]:
from flypipe.examples.pipeline.model.demo.config import config
from flypipe.examples.pipeline.model.demo.predict.predict import predict
import pandas as pd

config_df = pd.DataFrame(data=[{
    "artifact_location": ARTIFACT_LOCATION,
    "production_run_id":RUN_ID,
}])

df = predict.run(inputs={
    config: config_df
})

display(df)

### Predict using your own inputs?

In [ ]:
from flypipe.examples.pipeline.model.demo.config import config
from flypipe.examples.pipeline.model.demo.predict.predict import predict
from flypipe.examples.pipeline.model.demo.data import data
import pandas as pd

config_df = pd.DataFrame(data=[{
    "artifact_location": ARTIFACT_LOCATION,
    "production_run_id":RUN_ID,
}])

# Hard coded data to be predicted
df = pd.DataFrame(data = {
        'sepal_length': [6.6],
        'sepal_width': [3.1],
        'petal_length': [5.1],
        'petal_width': [2.4]
    })

# Run the predictions
predictions = (
    predict
    .run(inputs={
        config: config_df,
        data: df
    })
)

# Show predictions
display(predictions)